<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Pydantic/pydanticai_simple_with_tools_multiple_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -qU pydantic-ai nest-asyncio


In [4]:
import os
from google.colab import userdata
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['LOGFIRE_TOKEN'] = userdata.get('LOGFIRE_TOKEN')

In [3]:
import nest_asyncio
nest_asyncio.apply()

### getting started - a simple agent

In [31]:
from pydantic_ai import Agent
from pydantic import BaseModel

from pydantic_ai.models.gemini import GeminiModel


class ResponseFormat(BaseModel):
   thoughts: list[str]
   response: str

agent =Agent("gemini-2.0-flash-exp", system_prompt="You are a helpful assistant.", result_type=ResponseFormat)
result = await agent.run("Hello")

In [32]:
print(result)

RunResult(_all_messages=[ModelRequest(parts=[SystemPromptPart(content='You are a helpful assistant.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Hello', timestamp=datetime.datetime(2025, 1, 20, 15, 49, 47, 580863, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[ToolCallPart(tool_name='final_result', args=ArgsDict(args_dict={'response': 'Hello, how can I help?', 'thoughts': ['User said hello']}), tool_call_id=None, part_kind='tool-call')], timestamp=datetime.datetime(2025, 1, 20, 15, 49, 48, 135398, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='final_result', content='Final result processed.', tool_call_id=None, timestamp=datetime.datetime(2025, 1, 20, 15, 49, 48, 136539, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request')], _new_message_index=0, data=ResponseFormat(thoughts=['User said hello'], response='Hello, how can I help?'), _result_tool

In [35]:
result.data.response

'Hello, how can I help?'

In [36]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are a helpful assistant.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Hello', timestamp=datetime.datetime(2025, 1, 20, 15, 49, 47, 580863, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='final_result', args=ArgsDict(args_dict={'response': 'Hello, how can I help?', 'thoughts': ['User said hello']}), tool_call_id=None, part_kind='tool-call')], timestamp=datetime.datetime(2025, 1, 20, 15, 49, 48, 135398, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='final_result', content='Final result processed.', tool_call_id=None, timestamp=datetime.datetime(2025, 1, 20, 15, 49, 48, 136539, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request')]

In [10]:
history = []

user_input = input("User: ")

while user_input != "bye":

    print("User: ", user_input)

    result = await agent.run(user_prompt=user_input, message_history=history)

    print("Assistant: ", result.data)

    history = result.all_messages()

    user_input = input("User: ")


User: hello 
User:  hello 
Assistant:  Hello! How can I help you today?

User: I am Rizwan and I am a data scientist
User:  I am Rizwan and I am a data scientist
Assistant:  Hi Rizwan, it's nice to meet you! As a data scientist, you're working in a very exciting and in-demand field. What kind of work are you currently focusing on? Are you working on any interesting projects at the moment? I'd be happy to chat about data science with you, perhaps I can help with any questions you have or brainstorm ideas. Let me know what's on your mind!

User: write a story about me?
User:  write a story about me?
Assistant:  Okay, Rizwan, here's a story inspired by you being a data scientist. Let's say this is just one moment in your career, a snapshot in time:

**The Case of the Curious Correlations**

The flickering neon glow of Rizwan's monitor painted his face in shades of electric blue. Empty coffee cups, casualties of late nights and intense focus, littered his desk. Tonight, the air in his smal

### agent with dynamic system prompt

In [12]:
from dataclasses import dataclass
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.gemini import GeminiModel

@dataclass
class UserDetails:
    name: str
    age: int

model = GeminiModel(model_name="gemini-2.0-flash-exp")

agent = Agent(model, deps_type=UserDetails)

@agent.system_prompt
def update_system_prompt(ctx: RunContext[UserDetails]):
    return f"""You are a helpful assistant. You always address the user by their name.
    Right now you are talking with Mr {ctx.deps.name} and he is {ctx.deps.age} years old."""


In [14]:
# deps = UserDetails(name="Rizwan", age=38)
# deps = UserDetails(name="Abdul", age=35)
deps = UserDetails(name="Salman", age=23)

result = await agent.run("Hai", deps=deps)

result.data


"Hello Mr. Salman, it's nice to meet you! How can I help you today?\n"

### agent with tools

In [15]:
from dataclasses import dataclass
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.gemini import GeminiModel
import datetime

@dataclass
class UserDetails:
    name: str
    age: int

model = GeminiModel(model_name="gemini-2.0-flash-exp")

agent = Agent(model, deps_type=UserDetails)

@agent.system_prompt
def update_system_prompt(ctx: RunContext[UserDetails]):
    return f"""You are a helpful assistant. You always address the user by their name.
    Right now you are talking with Mr {ctx.deps.name} and he is {ctx.deps.age} years old."""

@agent.tool
def get_bank_balance(ctx: RunContext[UserDetails]):
    """Use this tool to get user bank balance"""
    if ctx.deps.name == "Rizwan":
        return "Rizwan's current bank balance is $100000."
    elif ctx.deps.name == "Abdul":
        return "Abdul's current bank balance is $200000."
    elif ctx.deps.name == "Salman":
        return "Salman's current bank balance is $300000."
    else:
        return "Please check the user name"

@agent.tool_plain
def get_current_date():
    return f"Today is {datetime.now()}"


In [18]:
deps = UserDetails(name="Rizwan", age=38)

result = await agent.run("What is my current bank balance?", deps=deps)
# result = await agent.run("What is today's date?", deps=deps)

result.data


'Your current bank balance is $100000, Mr. Rizwan.\n'

### multiple agents

In [19]:
hr_system_prompt = """
**You are an expert HR assistant with in-depth knowledge of HR policies, recruitment processes, employee management, and performance review criteria.**

1. **Company Policies:**
   - Working hours: 9 AM to 6 PM.
   - Annual leave: 20 days.
   - Sick leave: 12 days per year.
   - Remote work policy: Available twice a week for eligible employees.

2. **Recruitment Process:**
   - Application screening: Within 5 business days.
   - Interview rounds: Two technical interviews followed by one HR interview.
   - Offer letter issuance: Within 3 business days after the final interview.

3. **Performance Review Criteria:**
   - Key Performance Indicators (KPIs): Timeliness, collaboration, and quality of work.
   - Review frequency: Bi-annual.
   - Promotion policy: Based on performance scores and manager recommendations.
"""

sales_system_prompt = """
**You are a world-class AI Sales Agent skilled in personalized communication, lead generation, and closing deals. Your primary focus is assisting with the following tasks:**

1. **Product Offerings:**
   - Product A: AI-powered analytics software for small businesses.
   - Product B: Cloud-based CRM with automation features.
   - Product C: On-premise data security solutions for enterprises.

2. **Sales Pitch Highlights:**
   - Unique Selling Proposition (USP): "Our AI-driven solutions reduce operational costs by 30% and improve efficiency by 50%."
   - Current discount: 20% off annual subscriptions for Product A.
   - Free trial period: 30 days with full support.

3. **Meeting Scheduling:**
   - Available slots: Monday to Friday, 10 AM to 4 PM.
   - Duration: 30 minutes for product demos or consultations.

4. **Follow-up Process:**
   - First follow-up: 24 hours after the initial contact.
   - Second follow-up: After 3 business days with a personalized email.
"""

In [20]:
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai import Agent, RunContext
from dataclasses import dataclass

@dataclass
class MyDep:
    name: str
    hr_agent: Agent[None, str]
    sales_agent: Agent[None, str]

model = GeminiModel("gemini-2.0-flash-exp")

hr_agents = Agent(
    model=model,
    system_prompt=hr_system_prompt,
    retries=2,
)

sales_agents = Agent(
    model=model,
    system_prompt=sales_system_prompt,
    retries=2,
)
ceo_agent = Agent(
    model=model,
    deps_type=MyDep,
    system_prompt="You are a helpful assistant who always addresses the users by their name",
    retries=2,
)

@ceo_agent.system_prompt
def add_user_name(ctx: RunContext[MyDep]):
    return f"Right now you are talking with {ctx.deps.name}."

@ceo_agent.tool
async def ask_hr_manager(ctx: RunContext[MyDep], thoughts: str, question: str):
    """Use this tool to communicate with the HR manager

    Args:
        thoughts: Your thoughts about your action
        question: The detailed question which you want to ask the HR Manager
    """
    print("Thoughts: ", thoughts)
    result = await ctx.deps.hr_agent.run(user_prompt= question)
    return result.data

@ceo_agent.tool
async def ask_sales_manager(ctx: RunContext[MyDep], thoughts: str, question: str):
    """Use this tool to communicate with the Sales manager

    Args:
        thoughts: Your thoughts about your action
        question: The detailed question which you want to ask the Sales Manager
    """
    print("Thoughts: ", thoughts)
    result = await ctx.deps.sales_agent.run(user_prompt=question)
    return result.data


In [23]:
message_history = []

deps = MyDep(name="Rizwan", hr_agent=hr_agents, sales_agent=sales_agents)

user_input = input("User: ")

while user_input != "bye":

    result = await ceo_agent.run(user_input, message_history=message_history, deps=deps)

    message_history = result.all_messages()

    print("Assistant: ", result.data)

    user_input = input("User: ")


User: hi
Assistant:  Hi Rizwan, how can I help you today?

User: what are the products?
Assistant:  I do not have access to the product catalog. Please ask the sales manager about the products.

User: what are the available slots?
Assistant:  I do not have access to the available slots. Please ask the sales manager about the available slots.

User: I need to clarify the sick leave policy for Rizwan
Thoughts:  The user wants to know about the sick leave policy.
Assistant:  Okay Rizwan, I've asked the HR manager about the sick leave policy for you. You are eligible for 12 days of sick leave per year. Is there anything else I can help you with?

User: what are the products are there?
Thoughts:  The user wants to know what products are available.
Assistant:  Okay Rizwan, I've asked the sales manager about the available products. Here is a list:

*   **Product A:** AI-powered analytics software for small businesses. This product helps you gain valuable insights from your data, leading to be

In [25]:
message_history

[ModelRequest(parts=[SystemPromptPart(content='You are a helpful assistant who always addresses the users by their name', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='Right now you are talking with Rizwan.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='hi', timestamp=datetime.datetime(2025, 1, 20, 15, 38, 10, 851844, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content='Hi Rizwan, how can I help you today?\n', part_kind='text')], timestamp=datetime.datetime(2025, 1, 20, 15, 38, 11, 407547, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[UserPromptPart(content='what are the products?', timestamp=datetime.datetime(2025, 1, 20, 15, 38, 22, 858878, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content='I do not have access to the product catalog. Please ask the sales manager about the products.\n',